In [1]:
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler, scale
from sklearn.preprocessing import normalize
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Add, LSTM, Activation, Flatten, Dropout, Lambda, Conv1D, Input, BatchNormalization
from keras.models import Model
import numpy as np
from keras.utils import to_categorical
import fix_yahoo_finance as yf
%matplotlib inline
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

C:\Users\alahghol\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\alahghol\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [16]:
Stock="BBVA"
#df = yf.download(Stock,'2001-10-06')
#df.to_csv(Stock+'.csv')
df=pd.read_csv(Stock+'.csv')
df.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
4350,2019-01-18,5.92,5.98,5.88,5.95,5.95,4240200
4351,2019-01-22,5.88,5.91,5.86,5.90,5.90,4892000
4352,2019-01-23,5.97,6.00,5.94,5.99,5.99,3911800
4353,2019-01-24,5.93,5.95,5.87,5.93,5.93,3311700
4354,2019-01-25,5.99,6.11,5.99,6.10,6.10,4431700


In [17]:
data=df
data['Close'].replace(0, np.nan, inplace=True)
data['Close'].fillna(method='ffill', inplace=True)
values = data[['Open'] + ['High'] + ['Low'] + ['Close'] + ['Volume']]
values.dropna(inplace=True)
values = values.astype('float32')
values=values[(values['Open']!=0)&(values['High']!=0)&(values['Low']!=0)&(values['Close']!=0)&(values['Volume']!=0)]
scaled = (values.values[1:,:]/values.values[:-1,:])
scaled = pd.DataFrame(scaled)
all_inf_or_nan = scaled.isin([np.inf, -np.inf, np.nan]).all(axis='columns')
scaled = scaled[~all_inf_or_nan]
scaled = scaled.values

end=20
seq_len=15
Y = values.iloc[1:,3] - values.iloc[1:,0]
X = scaled[-(seq_len+1):-1,:].reshape(1,seq_len,5)
Y = np.array(Y)
y = Y[-1]
for i in range(1,4127):
    X = np.append(X, scaled[-(seq_len+1)-i:-1-i,:].reshape(1,seq_len,5),axis=0)
    #y = np.append(y, scaled[-1-i,3])
    y = np.append(y, Y[-1-i])
X = X[::-1,:,:]
y = y[::-1]
y1 = np.empty([len(y)], dtype=np.float32)
for i in range(len(y)):
    if y[i] >= 0.0:
        y1[i] = 1.0
    else:
        y1[i] = 0.0
#y1=to_categorical(y1)
train_X = X[:-end]
train_y = y1[:-end]
train_X, train_y = shuffle(train_X, train_y, random_state = 128)
test_X = X[-end:]
test_y = y1[-end:]
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

C:\Users\alahghol\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


(4107, 15, 5) (4107,) (20, 15, 5) (20,)


In [21]:
model = XGBClassifier(max_depth=3, n_estimators=100)
model.fit(train_X.reshape(4107,seq_len*5), train_y)
pred_y = model.predict(test_X.reshape(20,seq_len*5))
accuracy = accuracy_score(test_y,pred_y)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 75.00%


C:\Users\alahghol\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [18]:
#clf = RandomForestClassifier(n_estimators=250, max_depth=80,random_state=0)
#clf.fit(train_X.reshape(4265,75), train_y)
N = KNeighborsClassifier(n_neighbors=350)
N.fit(train_X.reshape(4107,seq_len*5), train_y)
N.score(test_X.reshape(20,seq_len*5),test_y)

0.85

In [23]:
N.predict(test_X.reshape(20,seq_len*5))

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float32)

In [20]:
test_y

array([1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1.], dtype=float32)